`02_prompt_engineering/homework_pe_20260109.ipynb`파일에 작성하세요.

### 1. 과제 개요 (Overview)

- **목표:** 학교생활이나 일상생활에서 겪는 사소한 불편함이나 비효율적인 문제를 발견하고, LLM과 프롬프트 엔지니어링 기술을 활용하여 이를 해결하는 웹 애플리케이션(프로토타입)을 제작한다.
- **핵심 역량:** 문제 해결 능력, 프롬프트 엔지니어링(Prompt Engineering), 기초 AI 애플리케이션 구현 능력.

### 2. 단계별 수행 가이드 (Step-by-Step Guide)

**Step 1: 문제 발굴 및 정의 (Problem Definition)**

- 나의 일상이나 친구들이 겪는 불편함을 관찰한다. 거창한 문제가 아니어도 된다.
- *예시:*
    - "오늘 급식 메뉴를 보고 맛있는 조합인지 아닌지 판단하기 귀찮다."
    - "동아리 공지사항을 쓸 때마다 말투를 다듬는 게 어렵다."
    - "복잡한 가정통신문 내용을 3줄로 요약해서 보고 싶다."
- **과제:** 해결하고자 하는 문제를 한 문장으로 정의하고, 타겟 사용자(나 자신 혹은 친구들)를 설정한다.

**Step 2: 프롬프트 엔지니어링 (Prompt Engineering)**

- LLM이 문제를 해결하도록 지시하는 최적의 프롬프트를 설계한다.
- **필수 포함 요소:**
    - **Role (페르소나):** AI에게 역할 부여 (예: "너는 영양사이자 메뉴 추천 전문가야.")
    - **Context (맥락):** 상황 설명 (예: "고등학생들이 좋아하는 입맛을 기준으로 분석해야 해.")
    - **Task (지시 사항):** 구체적인 수행 작업 (예: "메뉴 리스트를 입력받으면, 10점 만점으로 점수를 매기고 그 이유를 설명해 줘.")
    - **Constraints (제약 조건):** 출력 형식 제한 (예: "반말을 사용하고, 이모지를 3개 이상 포함해.")
- **과제:** 다양한 프롬프트를 테스트해 보고, 가장 결과가 좋은 최종 프롬프트를 선정한다.

**Step 3: 앱 구현 (Implementation)**

- 선정된 프롬프트를 실제 작동하는 앱 형태로 만든다.
- **도구 추천:**
    - GPT Chat Playground
    - Python + Streamlit (간단한 파이썬 코드로 웹 UI 제작)
- **과제:** 사용자가 입력창에 텍스트를 넣으면, LLM이 처리하여 결과를 보여주는 인터페이스를 구현한다.

**Step 4: 테스트 및 개선 (Testing & Iteration)**

- 친구들에게 앱을 써보게 하고 피드백을 받는다.
- LLM이 엉뚱한 대답을 하는 경우(할루시네이션 등), 프롬프트를 수정하여 성능을 개선한다.

### 3. 예시 시나리오 (Example Scenarios)

| **구분** | **앱 이름** | **기능 설명** | **프롬프트 핵심 전략** |
| --- | --- | --- | --- |
| **생활** | **냉장고 파먹기 AI** | 냉장고에 남은 재료만 입력하면 레시피 추천 | "가진 재료만 써야 함", "자취생이 할 수 있는 쉬운 요리" 제약 조건 추가 |
| **학업** | **발표 대본 생성기** | 조사한 자료를 넣으면 발표용 대본으로 변환 | "3분 분량으로 맞춰줘", "청중의 흥미를 끌 수 있는 유머 포함" 지시 |
| **소통** | **'너 T야?' 탐지기** | 카톡 메시지를 입력하면 공감 능력이 부족한지 판별 | "공감형 대화 전문가" 페르소나 부여, 감정 분석 및 수정 제안 |

In [ ]:
# 1. 필요한 라이브러리 설치 (Streamlit, OpenAI)
!pip install -q streamlit openai

# 2. 터널링을 위한 도구 설치 (외부에서 접속할 링크 생성용)
!npm install localtunnel

In [ ]:
# ==========================================
# 👇 [필수] 여기에 본인의 API Key를 넣어주세요!
# OPENAI_API_KEY = "sk---" # 넣으면 깃 오류(푸시할 때 오류남)가 생겨서 안넣음 
# ==========================================

import os
import time
import subprocess
from threading import Timer

# 1. API Key 확인
if not OPENAI_API_KEY.startswith("sk-"):
    print("❌ 오류: 맨 윗줄에 올바른 API Key를 입력하지 않으셨습니다!")
else:
    print("⏳ 설치를 시작합니다... (약 1분 소요)")
    
    # 2. 필수 라이브러리 설치
    !pip install -q streamlit openai

    # 3. Cloudflare Tunnel 다운로드 (비밀번호 없는 연결 도구)
    !wget -q -O cloudflared https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
    !chmod +x cloudflared

    # 4. 앱 파일(app.py) 생성
    app_code = f"""
import streamlit as st
from openai import OpenAI

# 페이지 기본 설정
st.set_page_config(page_title="GPT 학습 도우미", layout="centered")

# CSS로 입력창 강제 활성화 및 스타일 보정
st.markdown(
    '''
    <style>
    .stTextArea textarea {{ font-size: 16px !important; }}
    </style>
    ''', 
    unsafe_allow_html=True
)

st.title("🤖 GPT-4o-mini 학습 도우미")
st.info("비밀번호 없이 바로 접속되었습니다! 편하게 질문하세요.")

# API Key 설정
client = OpenAI(api_key="{OPENAI_API_KEY}")

# 입력 폼 (Form) - 엔터 키 전송 지원 및 새로고침 방지
with st.form("my_form"):
    text = st.text_area("질문 또는 공부할 내용 입력:", height=150, placeholder="예: 양자역학을 초등학생도 이해하게 설명해줘")
    submitted = st.form_submit_button("질문 보내기 🚀", type="primary")

if submitted:
    if not text:
        st.warning("⚠️ 내용을 입력해주세요.")
    else:
        try:
            with st.spinner("AI가 답변을 생각 중입니다..."):
                response = client.chat.completions.create(
                    model="gpt-4o-mini",
                    messages=[
                        {{"role": "system", "content": "당신은 도움이 되는 AI 튜터입니다."}},
                        {{"role": "user", "content": text}}
                    ]
                )
                st.success("답변이 도착했습니다!")
                st.markdown(response.choices[0].message.content)
        except Exception as e:
            st.error(f"오류가 발생했습니다: {{e}}")
"""
    with open("app.py", "w", encoding="utf-8") as f:
        f.write(app_code)

    print("✅ 설치 및 파일 생성 완료!")
    print("🚀 앱을 실행합니다... 잠시만 기다려주세요.")
    
    # 5. 백그라운드에서 Streamlit 실행
    subprocess.Popen(["streamlit", "run", "app.py", "--server.port", "8501"])
    time.sleep(3) # 실행 대기

    # 6. Cloudflare Tunnel 실행 (로그 파일에 링크 저장)
    with open('tunnel_log.txt', 'w') as log_file:
        tunnel_process = subprocess.Popen(
            ["./cloudflared", "tunnel", "--url", "http://localhost:8501"],
            stdout=log_file,
            stderr=log_file
        )

    # 7. 링크 추출 및 출력
    time.sleep(5) # 터널 생성 대기
    found_link = False
    print("\n👇 아래 링크를 클릭하세요 (비밀번호 없음!) 👇\n")
    
    # 로그 파일에서 trycloudflare.com 링크 찾기
    with open('tunnel_log.txt', 'r') as f:
        for line in f:
            if "trycloudflare.com" in line:
                # 링크만 깔끔하게 잘라내기
                import re
                link = re.search(r'https://[a-zA-Z0-9-]+\.trycloudflare\.com', line)
                if link:
                    print(f"🔗 접속 링크: {link.group(0)}")
                    found_link = True
                    break
    
    if not found_link:
        print("⚠️ 링크를 찾는 중입니다. 5초 뒤에 다시 시도합니다...")
        time.sleep(5)
        with open('tunnel_log.txt', 'r') as f:
             for line in f:
                if "trycloudflare.com" in line:
                    import re
                    link = re.search(r'https://[a-zA-Z0-9-]+\.trycloudflare\.com', line)
                    if link:
                        print(f"🔗 접속 링크: {link.group(0)}")